# Qiskit QVE - SU(2) Matrix Model [Lambda = 4]

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
from qiskit.circuit.library import TwoLocal, EfficientSU2

import sys
sys.path.append('../../utility')
from vqe_run import *
from qc_ansatze import *

# Convert the Hamiltonian to strings of Pauli ops

In [2]:
# Load the H file already created using qiskit 0.28 from MBP 2015
file = open("../../utility/pauliH_L4_g0.5.txt", "r")
content = file.read()
#print(content)

In [3]:
data = content.split(',\n')
data[0] = data[0].split('\n')[1]
data[-1] = data[-1].split('\n')[0]

data_coeff = []
data_str = []
for i in range(len(data)):
    data_coeff.append(float(data[i].split('*')[0]))
    data_str.append(data[i].split('*')[1].split(' ')[1])
    
Hpauli = list(zip(data_str,data_coeff))
#Hpauli

In [4]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyEigensolver
H4q = SparsePauliOp.from_list(Hpauli)

solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],5)
E_exact

Exact Result of qubit hamiltonian: [3.29894363 4.4638045  4.4638045  4.4638045 ]


3.29894

# Circuit ansatze

In [5]:
#CIRCUIT ANSATZ
from qiskit.circuit.library import TwoLocal, EfficientSU2

In [6]:
reps=1
ansatz_0a = TL_ansatz(H4q.num_qubits, 'ry', 'crx', "circular", reps)
ansatz_0b = TL_ansatz(H4q.num_qubits, 'rz', 'crx', 'circular', reps)
ansatz_0c = TL_ansatz(H4q.num_qubits, ['ry','rz'], 'crx', 'circular', reps)
ansatz_0d = TL_ansatz(H4q.num_qubits,['ry','y'], 'crx', 'circular', reps)

ansatz_1a =  TL_ansatz(H4q.num_qubits, 'ry', 'crx', "full", reps)
ansatz_1b =  TL_ansatz(H4q.num_qubits, 'rz', 'crx', "full", reps)
ansatz_1c =  TL_ansatz(H4q.num_qubits, ['ry','rz'], 'crx', "full", reps)
ansatz_1d =  TL_ansatz(H4q.num_qubits, ['ry','y'], 'crx', "full", reps)

Circuit ansatz with 36 parameters
Circuit ansatz with 36 parameters
Circuit ansatz with 60 parameters
Circuit ansatz with 36 parameters
Circuit ansatz with 90 parameters
Circuit ansatz with 90 parameters
Circuit ansatz with 114 parameters
Circuit ansatz with 90 parameters


In [7]:
reps=1
ansatz_0a_su2 = ef_ansatz(H4q.num_qubits, 'ry',  "circular", reps)
ansatz_0b_su2 = ef_ansatz(H4q.num_qubits, 'rz',  'circular', reps)
ansatz_0c_su2 = ef_ansatz(H4q.num_qubits, ['ry','rz'], 'circular', reps)
ansatz_0d_su2 = ef_ansatz(H4q.num_qubits,['ry','y'], 'circular', reps)

ansatz_1a_su2 =  ef_ansatz(H4q.num_qubits, 'ry',  "full", reps)
ansatz_1b_su2 =  ef_ansatz(H4q.num_qubits, 'rz',  "full", reps)
ansatz_1c_su2 =  ef_ansatz(H4q.num_qubits, ['ry','rz'], "full", reps)
ansatz_1d_su2 =  ef_ansatz(H4q.num_qubits, ['ry','y'],  "full", reps)

Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 48 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 48 parameters
Circuit ansatz with 24 parameters


# Qiskit QVE: Noiseless simulator

https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.primitives.Estimator.html

https://qiskit-community.github.io/qiskit-algorithms/tutorials/03_vqe_simulation_with_noise.html

In [8]:
#VQE ALGORITHMS
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals 
from qiskit_aer.primitives import Estimator as AerEstimator

import warnings
warnings.filterwarnings("ignore")
seed = 170
iterations = 350
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.5f}")
    return result

# COBYLA: EffSU2

In [9]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0a_su2)
counts_0a_c = counts
values_0a_c = values

VQE result: 4.18830


In [10]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0b_su2)
counts_0b_c = counts
values_0b_c = values

VQE result: 3.44775


In [11]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0c_su2)
counts_0c_c = counts
values_0c_c = values

VQE result: 7.59245


In [12]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0d_su2)
counts_0d_c = counts
values_0d_c = values

VQE result: 3.71764


In [13]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1a_su2)
counts_1a_c = counts
values_1a_c = values

VQE result: 3.94481


In [14]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1b_su2)
counts_1b_c = counts
values_1b_c = values

VQE result: 3.44775


In [15]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1c_su2)
counts_1c_c = counts
values_1c_c = values

VQE result: 3.90495


In [16]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1d_su2)
counts_1d_c = counts
values_1d_c = values

VQE result: 3.58207


In [17]:
df1 = pd.DataFrame({'values_0a_su2': values_0a_c})
df2 = pd.DataFrame({'values_0b_su2': values_0b_c})
df3 = pd.DataFrame({'values_0c_su2': values_0c_c})
df4 = pd.DataFrame({'values_0d_su2': values_0d_c})
df5 = pd.DataFrame({'values_1a_su2': values_1a_c})
df6 = pd.DataFrame({'values_1b_su2': values_1b_c})
df7 = pd.DataFrame({'values_1c_su2': values_1c_c})
df8 = pd.DataFrame({'values_1d_su2': values_1d_c})
qve_l2_tl_cobyla = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8], axis = 1)
qve_l2_tl_cobyla.to_csv('qve_l4_l=0.5_effsu2_cobyla.csv')
qve_l2_tl_cobyla

,values_0a_su2,values_0b_su2,values_0c_su2,values_0d_su2,values_1a_su2,values_1b_su2,values_1c_su2,values_1d_su2
0,14.750779,3.44775,14.288538,10.591751,14.532137,3.44775,15.672286,13.186027
1,16.128914,3.44775,14.349223,10.448450,15.485126,3.44775,16.264204,12.772526
2,15.319050,3.44775,15.003267,11.076336,15.072777,3.44775,16.520391,12.153789
3,14.985677,3.44775,15.062032,10.653701,15.547858,3.44775,14.474095,12.159995
4,14.011502,3.44775,14.594043,11.476472,14.676739,3.44775,14.582644,11.182701
...,...,...,...,...,...,...,...,...
345,NaN,NaN,7.730394,NaN,3.944915,NaN,3.944368,NaN
346,NaN,NaN,7.947095,NaN,3.953746,NaN,3.991980,NaN
347,NaN,NaN,7.792615,NaN,3.961443,NaN,4.003661,NaN
348,NaN,NaN,7.760387,NaN,3.955588,NaN,4.079410,NaN


# COBYLA: TwoLocal

In [18]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0a)
counts_0a_c = counts
values_0a_c = values

VQE result: 3.21510


In [19]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0b)
counts_0b_c = counts
values_0b_c = values

VQE result: 3.44775


In [20]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0c)
counts_0c_c = counts
values_0c_c = values

VQE result: 3.44960


In [21]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0d)
counts_0d_c = counts
values_0d_c = values

VQE result: 3.25823


In [22]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1a)
counts_1a_c = counts
values_1a_c = values

VQE result: 3.50693


In [23]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1b)
counts_1b_c = counts
values_1b_c = values

VQE result: 3.44775


In [24]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1c)
counts_1c_c = counts
values_1c_c = values

VQE result: 3.91300


In [25]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1d)
counts_1d_c = counts
values_1d_c = values

VQE result: 3.40591


In [26]:
df1 = pd.DataFrame({'values_0a': values_0a_c})
df2 = pd.DataFrame({'values_0b': values_0b_c})
df3 = pd.DataFrame({'values_0c': values_0c_c})
df4 = pd.DataFrame({'values_0d': values_0d_c})
df5 = pd.DataFrame({'values_1a': values_1a_c})
df6 = pd.DataFrame({'values_1b': values_1b_c})
df7 = pd.DataFrame({'values_1c': values_1c_c})
df8 = pd.DataFrame({'values_1d': values_1d_c})
qve_l2_tl_cobyla = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8], axis = 1)
qve_l2_tl_cobyla.to_csv('qve_l4_l=0.5_twolocal_cobyla.csv')
qve_l2_tl_cobyla

,values_0a,values_0b,values_0c,values_0d,values_1a,values_1b,values_1c,values_1d
0,10.021694,3.44775,16.659002,19.751946,15.881292,3.44775,16.792266,17.279504
1,10.557408,3.44775,16.708001,19.702168,16.253295,3.44775,15.944199,17.653205
2,11.059516,3.44775,16.971964,18.702041,15.588436,3.44775,17.570025,17.173515
3,9.886148,3.44775,16.645237,18.456878,15.854681,3.44775,16.185493,16.570318
4,10.606916,3.44775,16.597715,18.291057,16.594315,3.44775,14.646973,17.797441
...,...,...,...,...,...,...,...,...
345,3.225786,NaN,3.651052,3.310005,3.682795,3.44775,4.312475,3.437016
346,3.234796,NaN,3.601176,3.326705,3.536259,3.44775,4.078619,3.525480
347,3.220041,NaN,3.497693,3.258231,3.614882,3.44775,4.055003,3.756026
348,3.218187,NaN,3.729361,3.285829,3.506931,3.44775,4.000628,3.474188


# SPSA: EffSU2

In [27]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0a_su2)
counts_0a = counts
values_0a = values

VQE result: 3.61873


In [28]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0b_su2)
counts_0b = counts
values_0b = values

VQE result: 3.44775


In [29]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0c_su2)
counts_0c = counts
values_0c = values

VQE result: 5.96494


In [30]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0d_su2)
counts_0d = counts
values_0d = values

VQE result: 3.66239


In [31]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1a_su2)
counts_1a = counts
values_1a = values

VQE result: 3.40536


In [32]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1b_su2)
counts_1b = counts
values_1b = values

VQE result: 3.44775


In [33]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1c_su2)
counts_1c = counts
values_1c = values

VQE result: 4.53325


In [34]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1d_su2)
counts_1d = counts
values_1d = values

VQE result: 3.55380


In [35]:
# save the data to a dataframe 
list_l2 = {'values_0a_su2': values_0a, 'values_0b_su2': values_0b, 
        'values_0c_su2': values_0c, 'values_0d_su2': values_0d,
        'values_1a_su2': values_1a, 'values_1b_su2': values_1b, 
        'values_1c_su2': values_1c,'values_1d_su2': values_1d}
qve_l2_efsu2_spsa= pd.DataFrame(list_l2)
qve_l2_efsu2_spsa.to_csv('qve_l4_l=0.5_effsu2_spsa.csv')
qve_l2_efsu2_spsa

,values_0a_su2,values_0b_su2,values_0c_su2,values_0d_su2,values_1a_su2,values_1b_su2,values_1c_su2,values_1d_su2
0,14.342516,3.266858,17.379072,18.898924,15.739712,3.266858,15.700988,15.173515
1,15.349428,3.197083,16.191286,17.189545,17.208785,3.197083,16.259583,16.127295
2,15.251333,3.457357,16.944198,18.709785,17.490030,3.457357,15.699062,16.454812
3,15.358566,3.529644,16.296789,17.582094,16.308270,3.529644,15.901311,14.395045
4,15.307443,3.312718,16.852700,18.501784,16.194279,3.312718,14.845878,15.025581
...,...,...,...,...,...,...,...,...
746,3.627737,3.448488,6.233489,3.764151,3.456211,3.448488,4.697907,3.766980
747,3.663467,3.433968,6.443954,3.996202,3.631243,3.433968,4.514532,3.979113
748,3.674525,3.448488,6.299620,3.771617,3.693048,3.448488,4.643634,3.538909
749,3.741596,3.433968,6.287675,3.910079,3.682644,3.433968,4.654544,3.881095


# SPSA: TwoLocal

In [36]:
#3.18339
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0a)
counts_0a = counts
values_0a = values

VQE result: 3.49978


In [37]:
#3.17910
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0b)
counts_0b = counts
values_0b = values

VQE result: 3.44775


In [38]:
#3.62318
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0c)
counts_0c = counts
values_0c = values

VQE result: 3.39842


In [39]:
#3.13679
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0d)
counts_0d = counts
values_0d = values

VQE result: 3.38926


In [40]:
#3.16617
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1a)
counts_1a = counts
values_1a = values

VQE result: 3.32111


In [41]:
#3.17910
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1b)
counts_1b = counts
values_1b = values

VQE result: 3.44775


In [42]:
#4.10208
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1c)
counts_1c = counts
values_1c = values

VQE result: 3.50001


In [43]:
#4.26060
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1d)
counts_1d = counts
values_1d = values

VQE result: 3.30641


In [44]:
# save the data to a dataframe 
#don't reevaluate on a new run with just the new ansatze added
import pandas as pd
list_l2 = {'values_0a': values_0a, 'values_0b': values_0b, 
        'values_0c': values_0c, 'values_0d': values_0d,
        'values_1a': values_1a, 'values_1b': values_1b, 
        'values_1c': values_1c,'values_1d': values_1d}
qve_l2_tl_spsa= pd.DataFrame(list_l2)
qve_l2_tl_spsa.to_csv('qve_l4_l=0.5_twolocal_spsa.csv')
qve_l2_tl_spsa

,values_0a,values_0b,values_0c,values_0d,values_1a,values_1b,values_1c,values_1d
0,14.099404,3.266858,13.114340,13.687973,13.282304,3.266858,14.343590,17.031068
1,14.106894,3.197083,13.772049,15.175756,12.524710,3.197083,13.061766,16.911781
2,12.810528,3.457357,12.967693,15.441599,13.351950,3.457357,13.184383,17.020633
3,15.443475,3.529644,13.112561,13.508407,12.612290,3.529644,13.244885,16.519981
4,14.793133,3.312718,12.688761,15.698204,13.125106,3.312718,13.339208,15.706608
...,...,...,...,...,...,...,...,...
746,3.669880,3.448488,3.517688,3.591699,3.516934,3.448488,3.772713,3.570344
747,3.336743,3.433968,3.463410,3.487190,3.405150,3.433968,3.706335,3.355426
748,3.739483,3.448488,3.552579,3.632985,3.491169,3.448488,3.758648,3.604016
749,3.512323,3.433968,3.521838,3.449686,3.440867,3.433968,3.455018,3.554500
